# 1) Carga del dataset definitivo y diagnóstico base


**Objetivo:** asegurar que partimos de `df_final` (290×~58), ver tipos de datos y el panorama de nulos antes de decidir reglas de imputación.

**Qué comprobaremos:**
- Fuente cargada (CSV preferente).
- `shape` y distribución de `Country`.
- Conteo de tipos (`float`, `int`, `object`).
- Listas de columnas numéricas y categóricas.
- Top 15 variables con mayor % de nulos.

In [1]:
from pathlib import Path
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

# Rutas esperadas
out_dir = Path("outputs")
csv_path  = out_dir / "dataset_final.csv"
xlsx_path = out_dir / "dataset_final.xlsx"

print("¿Existe CSV?:", csv_path.exists(), "→", csv_path.resolve())
print("¿Existe XLSX?:", xlsx_path.exists(), "→", xlsx_path.resolve())

# Carga (prioriza CSV)
if csv_path.exists():
    df = pd.read_csv(csv_path)
    fuente = "CSV"
elif xlsx_path.exists():
    df = pd.read_excel(xlsx_path, sheet_name="master")
    fuente = "Excel"
else:
    raise FileNotFoundError("No encuentro dataset_final.* en 'outputs/'")

print("\nFuente utilizada:", fuente)
print("Shape:", df.shape)

# Country
if "Country" in df.columns:
    print("\nFrecuencias 'Country':")
    print(df["Country"].value_counts(dropna=False))
else:
    print("\n⚠️ No se encontró 'Country' en el DataFrame.")

# Tipos
print("\nConteo de tipos:")
print(df.dtypes.value_counts())

# Listas de columnas por tipo (sencillo)
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
categorical_cols = [c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])]

print("\n# Numéricas:", len(numeric_cols))
print(numeric_cols)
print("\n# Categóricas:", len(categorical_cols))
print(categorical_cols)

# Nulos: % por columna (top 15)
null_perc = (df.isna().sum() / len(df) * 100).round(2).sort_values(ascending=False)
print("\nTop 15 columnas por % de nulos:")
display(null_perc.head(15).to_frame(name="%_nulos"))


¿Existe CSV?: True → C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_final.csv
¿Existe XLSX?: True → C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_final.xlsx

Fuente utilizada: CSV
Shape: (290, 58)

Frecuencias 'Country':
Country
Brazil    143
Spain      77
Mexico     70
Name: count, dtype: int64

Conteo de tipos:
float64    45
object     12
int64       1
Name: count, dtype: int64

# Numéricas: 46
['Age (years)', 'Time of disease (years)', 'HCQ use (mg/day)', 'SLICC', 'SLEDAI', 'Weight (kg)', 'Height (m)', 'BMI (kg/m2)', 'Waist Circ (cm)', 'Systolic Blood Pressure (mm/Hg)', 'Diastolic Blood Pressure (mm/Hg)', 'Glucose (mg/dL)', 'Total cholesterol (mg/dL)', 'LDL (mg/dL)', 'HDL (mg/dL)', 'Triglycerides (mg/dL)', 'Albumin (g/dL)', 'GOT_AST (U/L)', 'GPT_ALT (U/L)', 'Urea (mg/dL)', 'Creatinine (mg/dL)', 'Folic acid (ng/mL)', 'Vitamin B12 (ng/ml)', 'Leukocytes', 'Neutrophils', 'Lymphocytes', 'Monocytes', 'Platelets', 'Hemoglobin', 'Hematocrit', 'VCM', 'CHCM', 'RDW', 'VSG 

,%_nulos
MCS12 (HRQoL),73.45
PCS12 (HRQoL),73.45
METs-min/week,73.45
VSG (mm),53.79
SLICC,53.79
HCQ use (mg/day),49.31
Vitamin B12 (ng/ml),45.86
FACIT Fatigue Scale,41.72
IPAQ,34.48
Lipid (%TEI),29.66


# 2) Plan de imputación (sin ejecutarla aún)



**Objetivo:** definir qué variables imputaremos y con qué método, y cuáles NO imputaremos (por criterio clínico o porque requieren revisión de tipo).

## Reglas que seguiremos
- **NO imputar (por ahora)**:
  - *Exploratorias clave con >50% nulos:* `PCS12 (HRQoL)`, `MCS12 (HRQoL)`, `METs-min/week`.
  - *Clínicas relevantes pero con >50% nulos:* `SLICC`, `VSG (mm)` → se analizarán en subconjuntos.
  - *Fármacos (dosis):* `HCQ use (mg/day)` → una dosis faltante no es “0”; mejor no inventarla.

- **Numéricas → Mediana por país (`Country`)**  
  Justificación: robusta a outliers y respeta diferencias México/Brasil/España.  
  Ejemplos: analíticas (glucosa, lípidos, creatinina…), antropometría (IMC, cintura), dieta (kcal, g/día, %TEI), marcadores (C3, TyG…), etc.  
  *Criterio:* se imputan las que tienen **≤50% nulos**.

- **Categóricas → Moda por país (`Country`)**  
  Ejemplos: `Smoking habits`, `Marital status`, `Education level`, `Race`, `IPAQ`, `Anti-dsDNA` (si es cualitativa).  
  *Criterio:* solo si tienen huecos (y son realmente categóricas).

- **Revisión de tipo (no imputar todavía):**  
  Variables que aparecen como `object` pero *deberían* ser numéricas (p. ej. `Vitamin D (ng/mL)`, `C-reactive protein`, `C4 complement`).  
  Las revisaremos aparte para convertirlas a numéricas con seguridad antes de decidir imputación.


In [2]:
import pandas as pd

# 1) Columnas por tipo (según tu salida)
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
categorical_cols = [c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])]

# 2) Grupos que NO vamos a imputar (por criterio)
no_impute = [
    "PCS12 (HRQoL)", "MCS12 (HRQoL)", "METs-min/week",  # exploratorias con muchos nulos
    "SLICC", "VSG (mm)",                               # clínicas relevantes con >50%
    "HCQ use (mg/day)"                                  # dosis farmacológica
]
no_impute = [c for c in no_impute if c in df.columns]

# 3) Variables "a revisar tipo" (aparecen como categóricas pero deberían ser numéricas)
type_review = [c for c in ["Vitamin D (ng/mL)", "C-reactive protein", "C4 complement"] if c in df.columns]

# 4) Candidatas a imputación por mediana (numéricas ≤50% nulos, excluyendo las de no_impute)
null_perc = (df.isna().sum() / len(df) * 100)
median_by_country = [
    c for c in numeric_cols
    if (c not in no_impute)
    and (null_perc.get(c, 0) > 0)
    and (null_perc.get(c, 0) <= 50)
]

# 5) Candidatas a imputación por moda (categóricas con nulos, excluyendo Country y las de revisión de tipo)
mode_by_country = [
    c for c in categorical_cols
    if c not in (["Country"] + type_review)
    and null_perc.get(c, 0) > 0
]

# 6) Resumen del plan (solo mostramos conteos y ejemplos)
print("PLAN DE IMPUTACIÓN (previo, sin aplicar):\n")

print("No imputar (criterio clínico/exploratorio):", len(no_impute))
print(no_impute, "\n")

print("Revisión de tipo (no imputar aún):", len(type_review))
print(type_review, "\n")

print("Numéricas → Mediana por Country (≤50% nulos):", len(median_by_country))
print(median_by_country[:12], ("..." if len(median_by_country) > 12 else ""), "\n")

print("Categóricas → Moda por Country:", len(mode_by_country))
print(mode_by_country, "\n")

# Comprobación de cobertura (¿cuántas columnas con nulos quedan fuera del plan?)
cols_with_nulls = [c for c in df.columns if null_perc.get(c, 0) > 0]
covered = set(no_impute) | set(type_review) | set(median_by_country) | set(mode_by_country)
uncovered = [c for c in cols_with_nulls if c not in covered]

print("Columnas con nulos NO cubiertas por el plan (deberían ser 0 o casos muy especiales):", len(uncovered))
print(uncovered)


PLAN DE IMPUTACIÓN (previo, sin aplicar):

No imputar (criterio clínico/exploratorio): 6
['PCS12 (HRQoL)', 'MCS12 (HRQoL)', 'METs-min/week', 'SLICC', 'VSG (mm)', 'HCQ use (mg/day)'] 

Revisión de tipo (no imputar aún): 3
['Vitamin D (ng/mL)', 'C-reactive protein', 'C4 complement'] 

Numéricas → Mediana por Country (≤50% nulos): 38
['Time of disease (years)', 'SLEDAI', 'Height (m)', 'BMI (kg/m2)', 'Waist Circ (cm)', 'Systolic Blood Pressure (mm/Hg)', 'Diastolic Blood Pressure (mm/Hg)', 'Glucose (mg/dL)', 'Total cholesterol (mg/dL)', 'LDL (mg/dL)', 'HDL (mg/dL)', 'Triglycerides (mg/dL)'] ... 

Categóricas → Moda por Country: 6
['Race', 'Education level', 'Smoking habits', 'Anti-dsDNA', 'Protein intake (%TEI)', 'IPAQ'] 

Columnas con nulos NO cubiertas por el plan (deberían ser 0 o casos muy especiales): 0
[]


## 2.1) Resumen del plan de imputación

**Objetivo:** Definir una estrategia clara, defendible y coherente para manejar los valores faltantes.

**Criterios aplicados:**

1. **No imputar (6 variables)**  
   - `PCS12 (HRQoL)`, `MCS12 (HRQoL)`, `METs-min/week`: variables de calidad de vida y actividad física, con >70% nulos.  
     Se mantienen para análisis exploratorios en subconjuntos.  
   - `SLICC`, `VSG (mm)`: índices clínicos importantes pero incompletos (>50% nulos). Se conservan con nulos.  
   - `HCQ use (mg/day)`: dosis farmacológica; un valor faltante no se puede inventar.

2. **Revisión de tipo (3 variables)**  
   - `Vitamin D (ng/mL)`, `C-reactive protein`, `C4 complement`: aparecen como categóricas (`object`) pero deberían ser numéricas.  
     Antes de imputar deben convertirse de forma segura (limpieza de comas, símbolos, etc.).

3. **Numéricas con ≤50% nulos → Mediana por país**  
   - 38 variables (ej. `Glucose (mg/dL)`, `BMI (kg/m2)`, `Total cholesterol (mg/dL)`, etc.).  
   - Justificación: la mediana es robusta a outliers y la imputación por país (`Country`) respeta las diferencias entre cohortes.

4. **Categóricas con nulos → Moda por país**  
   - 6 variables (ej. `Smoking habits`, `Education level`, `Race`, `IPAQ`).  
   - Justificación: la moda representa el valor más frecuente en cada país, reduciendo sesgos.

**Resultado:**  
Todas las columnas con nulos quedan cubiertas por este plan. No se detectan variables “huérfanas” sin estrategia definida.


# 3) Conversión segura a numérico (Vitamin D, CRP, C4)

**Objetivo**: transformar a `float` conservando decimales y evitando confusiones de comas/puntos.

**Qué hace el código**:
- Define `clean_to_numeric`: convierte strings tipo `12,3`, `1.234,56`, `45.8%` a `float`.
- Aplica la conversión a: `Vitamin D (ng/mL)`, `C-reactive protein`, `C4 complement` (si existen).
- Muestra un **informe**: tipo antes/después y cuántos valores numéricos hemos conseguido.

In [3]:
import re
import numpy as np
import pandas as pd

df_work = df.copy()  # trabajamos sobre una copia

cols_to_cast = [c for c in ["Vitamin D (ng/mL)", "C-reactive protein", "C4 complement"] if c in df_work.columns]

def clean_to_numeric(series: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(series):
        return series.astype(float)
    s = series.astype(str).str.strip()
    # 1) formato europeo con puntos de miles y coma decimal
    s = s.str.replace(r"\s", "", regex=True)
    s = s.str.replace("%", "", regex=False)
    # si hay punto y coma → asumimos 1.234,56 -> 1234.56
    mask_both = s.str.contains(r"\.") & s.str.contains(r",")
    s.loc[mask_both] = s.loc[mask_both].str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    # solo coma -> decimal
    mask_coma = ~mask_both & s.str.contains(",")
    s.loc[mask_coma] = s.loc[mask_coma].str.replace(",", ".", regex=False)
    # limpiar cualquier resto no numérico (excepto signo, punto, e/E)
    s = s.str.replace(r"[^0-9eE\+\-\.]", "", regex=True)
    s = s.replace({"": np.nan, ".": np.nan, "-": np.nan, "+": np.nan})
    return pd.to_numeric(s, errors="coerce")

cast_report = []
for c in cols_to_cast:
    before_dtype = df_work[c].dtype
    before_nonnull = df_work[c].notna().sum()
    df_work[c] = clean_to_numeric(df_work[c])
    after_dtype = df_work[c].dtype
    after_numeric = df_work[c].notna().sum()
    cast_report.append({
        "col": c,
        "dtype_antes": str(before_dtype),
        "no_nulos_antes": int(before_nonnull),
        "dtype_despues": str(after_dtype),
        "no_numericos_despues": int(after_numeric),
    })

pd.DataFrame(cast_report)


,col,dtype_antes,no_nulos_antes,dtype_despues,no_numericos_despues
0,Vitamin D (ng/mL),object,256,float64,245
1,C-reactive protein,object,282,float64,256
2,C4 complement,object,240,float64,232


In [4]:
for c in ["Vitamin D (ng/mL)", "C-reactive protein", "C4 complement"]:
    if c in df.columns:
        before = df[c]
        after  = df_work[c]
        perdidos = before[before.notna() & after.isna()]
        print(f"\n{c} → filas que eran texto y ahora son NaN (muestra 10):")
        print(perdidos.head(10))



Vitamin D (ng/mL) → filas que eran texto y ahora son NaN (muestra 10):
12    2025-03-23 00:00:00
14    2025-03-12 00:00:00
34    2025-07-25 00:00:00
44    2025-03-15 00:00:00
53    2025-12-27 00:00:00
57    2025-08-26 00:00:00
59    2025-08-26 00:00:00
63    2025-06-23 00:00:00
65    2025-07-15 00:00:00
66    2025-05-29 00:00:00
Name: Vitamin D (ng/mL), dtype: object

C-reactive protein → filas que eran texto y ahora son NaN (muestra 10):
3     2025-12-14 00:00:00
7     2025-07-01 00:00:00
8     2025-01-09 00:00:00
10    2025-04-01 00:00:00
11    2025-05-07 00:00:00
13    2025-09-05 00:00:00
17    2025-02-03 00:00:00
21    2025-01-02 00:00:00
23    2025-09-05 00:00:00
25    2025-04-01 00:00:00
Name: C-reactive protein, dtype: object

C4 complement → filas que eran texto y ahora son NaN (muestra 10):
13    2025-11-07 00:00:00
14    2025-07-20 00:00:00
19    2025-04-12 00:00:00
20    2025-05-10 00:00:00
40    2025-06-29 00:00:00
51    2025-02-22 00:00:00
55    2025-06-05 00:00:00
96    

# 4a) Imputación numérica por mediana estratificada por país




**Por qué mediana por `Country`:**
- La mediana es robusta a outliers.
- Respetamos posibles diferencias entre México/Brasil/España.

**Qué hace el código:**
1) Revisa % de nulos de `Vitamin D`, `CRP`, `C4` tras la conversión.
2) Elige todas las columnas **numéricas** con **0 < % nulos ≤ 50** (excluye las de `no_impute`).
3) Imputa con la **mediana dentro de cada país**.
4) Muestra un pequeño **reporte** con cuántos valores se rellenaron por columna.

In [5]:
import pandas as pd
import numpy as np

# 0) Listas de control
no_impute = [c for c in ["PCS12 (HRQoL)", "MCS12 (HRQoL)", "METs-min/week", "SLICC", "VSG (mm)", "HCQ use (mg/day)"] if c in df_work.columns]

# 1) Chequeo rápido de nulos en las 3 columnas recién convertidas
for c in ["Vitamin D (ng/mL)", "C-reactive protein", "C4 complement"]:
    if c in df_work.columns:
        na = df_work[c].isna().sum()
        print(f"{c}: nulos={na}  ({round(100*na/len(df_work),2)}%)")

# 2) Selección de numéricas imputables (≤50% nulos y no en no_impute)
null_perc_work = (df_work.isna().sum() / len(df_work) * 100)
numeric_cols = [c for c in df_work.columns if pd.api.types.is_numeric_dtype(df_work[c])]
median_cols = [
    c for c in numeric_cols
    if (c not in no_impute) and (0 < null_perc_work.get(c, 0) <= 50)
]

# 3) Imputación por mediana dentro de cada Country
log_num = []
for c in median_cols:
    n_before = int(df_work[c].isna().sum())
    df_work[c] = df_work.groupby("Country")[c].transform(lambda s: s.fillna(s.median()))
    n_after = int(df_work[c].isna().sum())
    log_num.append({"variable": c, "rellenos": n_before - n_after})

imput_num_report = pd.DataFrame(log_num).sort_values("rellenos", ascending=False)
print("\nImputación numérica — valores rellenados (top 20):")
display(imput_num_report.head(20))

# 4) Nulos restantes (vista rápida)
rest_nulls = (df_work.isna().sum() / len(df_work) * 100).round(2).sort_values(ascending=False)
print("\n% de nulos restante (top 15):")
display(rest_nulls.head(15).to_frame(name="%_nulos"))


Vitamin D (ng/mL): nulos=45  (15.52%)
C-reactive protein: nulos=34  (11.72%)
C4 complement: nulos=58  (20.0%)

Imputación numérica — valores rellenados (top 20):


,variable,rellenos
18,Folic acid (ng/mL),80
19,Vitamin B12 (ng/ml),63
32,C4 complement,58
40,FACIT Fatigue Scale,51
31,C3 complement,50
20,Vitamin D (ng/mL),45
3,BMI (kg/m2),44
24,Monocytes,38
13,Albumin (g/dL),34
12,C-reactive protein,34



% de nulos restante (top 15):


,%_nulos
METs-min/week,73.45
MCS12 (HRQoL),73.45
PCS12 (HRQoL),73.45
VSG (mm),53.79
SLICC,53.79
HCQ use (mg/day),49.31
IPAQ,34.48
Protein intake (%TEI),29.31
Lipid (%TEI),26.55
Energy intake (kcal/day),26.55


## 4b) Imputación categórica por moda estratificada por país

**Por qué moda por `Country`:**
- Representa el valor más frecuente dentro de cada cohorte.
- Evita asignar valores poco realistas para una población concreta.

**Qué hace el código:**
1. Selecciona las 6 variables categóricas con nulos.
2. Calcula la moda (valor más frecuente) en cada país.
3. Rellena los valores faltantes con esa moda.
4. Genera un reporte con cuántos valores fueron imputados por columna.


In [6]:
def mode_series(s: pd.Series):
    m = s.mode(dropna=True)
    return m.iloc[0] if len(m) else np.nan

mode_cols = ['Race', 'Education level', 'Smoking habits', 'Anti-dsDNA', 'Protein intake (%TEI)', 'IPAQ']
mode_cols = [c for c in mode_cols if c in df_work.columns]

log_cat = []
for c in mode_cols:
    n_before = df_work[c].isna().sum()
    modes = df_work.groupby("Country")[c].apply(mode_series)
    df_work[c] = df_work.groupby("Country")[c].transform(lambda s: s.fillna(modes.get(s.name, np.nan)))
    n_after = df_work[c].isna().sum()
    log_cat.append({"variable": c, "rellenos": int(n_before - n_after)})

imput_cat_report = pd.DataFrame(log_cat).sort_values("rellenos", ascending=False)
print("Imputación categórica — valores rellenados:")
display(imput_cat_report)

# Verificación final de nulos tras imputación categórica
remaining_nulls = (df_work.isna().sum() / len(df_work) * 100).round(2).sort_values(ascending=False)
print("\n% de nulos restantes (top 15):")
display(remaining_nulls.head(15).to_frame(name="%_nulos"))


Imputación categórica — valores rellenados:


C:\Users\manue\AppData\Local\Temp\ipykernel_41676\2213837357.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_work[c] = df_work.groupby("Country")[c].transform(lambda s: s.fillna(modes.get(s.name, np.nan)))


,variable,rellenos
5,IPAQ,100
4,Protein intake (%TEI),8
3,Anti-dsDNA,5
1,Education level,3
2,Smoking habits,2
0,Race,1



% de nulos restantes (top 15):


,%_nulos
MCS12 (HRQoL),73.45
PCS12 (HRQoL),73.45
METs-min/week,73.45
VSG (mm),53.79
SLICC,53.79
HCQ use (mg/day),49.31
Carbohydrate intake (g/day),26.55
Protein intake (%TEI),26.55
Energy intake (kcal/day),26.55
Carbohydrate intake (%TEI),26.55


### 4c) Conclusión de la imputación

- Se completó la imputación de todas las variables numéricas y categóricas con ≤50% de nulos.
- Se utilizó **mediana por país** para las variables numéricas y **moda por país** para las categóricas.
- Algunas variables con >50% nulos (PCS12, MCS12, METs-min/week, SLICC, VSG, HCQ use) **no se imputaron**, ya que resultaría poco fiable inventar valores clínicos.
- En la cohorte de España se observaron huecos sistemáticos (ej. IPAQ, dieta, escalas de calidad de vida).  
  → Estos se mantuvieron como nulos, ya que reflejan ausencia real de datos en la recogida.
- El dataset resultante es más completo y estable, pero conserva nulos en variables críticas que se analizarán en subconjuntos específicos.


# 5) Detección de outliers (IQR)



**Objetivo:** identificar valores extremos que puedan distorsionar el análisis.

**Método IQR (Interquartile Range):**
- Para cada variable numérica:
  - Q1 = percentil 25
  - Q3 = percentil 75
  - IQR = Q3 − Q1
  - Outlier = valor < Q1 − 1.5·IQR o > Q3 + 1.5·IQR
- Se calcula el nº y % de outliers por variable.

**Nota:** no se eliminan ni modifican datos; solo se documenta la cantidad de valores extremos.


In [7]:
num_cols = [c for c in df_work.columns if pd.api.types.is_numeric_dtype(df_work[c])]
outlier_info = []

for c in num_cols:
    s = df_work[c].dropna()
    if s.empty:
        continue
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        continue
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    mask = (s < lower) | (s > upper)
    n_out = mask.sum()
    outlier_info.append({
        "variable": c,
        "n_outliers": int(n_out),
        "%_outliers": round(100 * n_out / len(s), 2)
    })

outlier_summary = pd.DataFrame(outlier_info).sort_values("%_outliers", ascending=False)

print("Top 15 variables con más outliers:")
display(outlier_summary.head(15))


Top 15 variables con más outliers:


,variable,n_outliers,%_outliers
34,RDW,71,24.48
33,CHCM,71,24.48
22,Folic acid (ng/mL),64,22.07
40,Carbohydrate intake (g/day),44,20.66
32,VCM,57,19.66
43,Lipid intake (g/day),41,19.25
42,Protein intake (g/day),34,15.96
30,Hemoglobin,45,15.52
16,C-reactive protein,34,11.72
23,Vitamin B12 (ng/ml),20,9.09


## 5b) Informe de rangos por variable

**Objetivo:** comprobar que los valores extremos (mínimos y máximos) son clínicamente plausibles.  
Esto complementa la detección de outliers (%), ya que un valor fuera del IQR puede ser real, pero un máximo imposible (ej. 70 000 g/día de carbohidratos) debe revisarse.

**Qué muestra el código:**
- Para cada variable numérica: min, Q1, mediana, Q3 y max.
- Ordenado por el valor máximo descendente.


In [8]:
# Informe de rangos para todas las variables numéricas
range_report = df_work.describe(percentiles=[0.25, 0.5, 0.75]).T[
    ["min", "25%", "50%", "75%", "max"]
].sort_values("max", ascending=False)

print("Top 20 variables con máximos más altos (posibles errores a revisar):")
display(range_report.head(20))

print("\nEjemplo de variables con valores bajos (min):")
display(range_report.sort_values("min").head(10))


Top 20 variables con máximos más altos (posibles errores a revisar):


,min,25%,50%,75%,max
Carbohydrate intake (g/day),27.50,159.340000,225.800000,2.310270e+05,9903167.00
Protein intake (g/day),8.20,57.080000,81.720000,6.100900e+04,9850034.00
Lipid intake (g/day),11.90,54.483333,77.100000,4.284300e+04,9373367.00
Energy intake (kcal/day),325.00,1356.150000,1930.000000,1.144237e+06,7642856.00
Carbohydrate intake (%TEI),18.50,45.369893,51.000000,4.009231e+06,6987222.00
Lipid (%TEI),5.90,33.734277,38.223823,2.101677e+06,4933822.00
Hemoglobin,0.09,12.507500,13.500000,1.443000e+01,46005.00
Time of disease (years),0.25,4.000000,10.000000,1.500000e+01,45417.00
Urea (mg/dL),14.00,23.000000,30.000000,3.675000e+01,25278.00
Vitamin D (ng/mL),7.20,25.425000,32.895000,5.652500e+01,9795.00



Ejemplo de variables con valores bajos (min):


,min,25%,50%,75%,max
SLEDAI,0.00,0.0000,0.000,2.0000,17.0
SLICC,0.00,0.0000,1.000,6.0000,11.0
C-reactive protein,0.00,1.0000,2.450,4.5000,58.0
METs-min/week,0.00,396.0000,742.500,1554.0000,5736.0
Monocytes,0.00,0.3700,0.580,8.2175,15.0
Hemoglobin,0.09,12.5075,13.500,14.4300,46005.0
C4 complement,0.17,15.0000,17.525,22.9000,75.0
Leukocytes,0.17,4.3200,5.460,6.7350,20.1
Time of disease (years),0.25,4.0000,10.000,15.0000,45417.0
Lymphocytes,0.31,1.3625,2.145,29.4400,52.7


## 5c) Corrección de valores imposibles (rango clínico)

**Objetivo:** asegurar que no se mantienen valores irreales que distorsionen los análisis.

**Método aplicado:**
- Se definieron rangos clínicamente plausibles para cada variable clave.
- Valores fuera de ese rango se consideran errores de origen (ej. conversión de fechas, números gigantes).
- Estos valores se reemplazan por `NaN` para no contaminar los análisis.

**Ejemplos de rangos aplicados:**
- Hemoglobin: 5–25 g/dL  
- Time of disease (years): 0–80 años  
- Urea: 5–100 mg/dL  
- Vitamin D: 5–150 ng/mL  
- Folic acid: 2–50 ng/mL  
- RDW: 10–25 %  
- Dieta:  
  - Energy intake: 500–6000 kcal/día  
  - Macronutrientes: 10–600 g/día  
  - %TEI: 5–80 %  

**Nota:** valores extremos pero clínicamente plausibles (ej. triglicéridos >1000 mg/dL, colesterol >400 mg/dL) se mantuvieron.


In [10]:
# Versión robusta: convierte a numérico dentro del bucle antes de aplicar rangos
df_clean = df_work.copy()

ranges = {
    "Hemoglobin": (5, 25),
    "Time of disease (years)": (0, 80),
    "Urea (mg/dL)": (5, 100),
    "Vitamin D (ng/Ml)": (5, 150),          # ojo a mayúsculas/minúsculas del nombre real
    "Vitamin D (ng/mL)": (5, 150),          # incluyo ambas por seguridad
    "Folic acid (ng/mL)": (2, 50),
    "RDW": (10, 25),
    "Energy intake (kcal/day)": (500, 6000),
    "Carbohydrate intake (g/day)": (10, 600),
    "Protein intake (g/day)": (5, 300),
    "Lipid intake (g/day)": (5, 300),
    "Carbohydrate intake (%TEI)": (20, 70),
    "Protein intake (%TEI)": (5, 40),
    "Lipid (%TEI)": (15, 50),
}

corrections = []

for col, (low, high) in ranges.items():
    if col not in df_clean.columns:
        continue
    
    # 1) fuerza a numérico para poder comparar; lo que no sea número pasa a NaN
    s_num = pd.to_numeric(df_clean[col], errors="coerce")
    
    # 2) construye la máscara con la serie numérica
    mask_invalid = (s_num < low) | (s_num > high)
    n_invalid = int(mask_invalid.sum())
    
    # 3) aplica los NaN sobre la copia de trabajo SOLO donde es inválido
    before_na = int(df_clean[col].isna().sum())
    df_clean.loc[mask_invalid, col] = np.nan
    after_na = int(df_clean[col].isna().sum())
    
    corrections.append({
        "variable": col,
        "rango": f"[{low}, {high}]",
        "valores_invalidos": n_invalid,
        "nulos_antes": before_na,
        "nulos_despues": after_na
    })

correction_report = pd.DataFrame(corrections).sort_values("valores_invalidos", ascending=False)
print("Correcciones aplicadas:")
display(correction_report)


Correcciones aplicadas:


,variable,rango,valores_invalidos,nulos_antes,nulos_despues
5,RDW,"[10, 25]",71,0,71
12,Lipid (%TEI),"[15, 50]",71,77,148
10,Carbohydrate intake (%TEI),"[20, 70]",69,77,146
6,Energy intake (kcal/day),"[500, 6000]",66,77,143
7,Carbohydrate intake (g/day),"[10, 600]",66,77,143
8,Protein intake (g/day),"[5, 300]",66,77,143
11,Protein intake (%TEI),"[5, 40]",66,77,143
9,Lipid intake (g/day),"[5, 300]",65,77,142
4,Folic acid (ng/mL),"[2, 50]",64,0,64
0,Hemoglobin,"[5, 25]",40,0,40


## 5d) Micro-imputación final tras correcciones de rango

**Por qué:** en 5c recodificamos valores imposibles como `NaN`.  
**Qué haremos ahora:** rellenar, por **mediana por `Country`**, únicamente las variables numéricas con **0 < % nulos ≤ 50%**, excluyendo las de `no_impute`.  
Variables que quedaron con **>50% nulos** (p. ej., algunos %TEI) **no se imputan** y se analizarán por subconjuntos.


In [14]:
# Lista de exclusión (no se imputan nunca)
no_impute = [c for c in ["PCS12 (HRQoL)", "MCS12 (HRQoL)", "METs-min/week", "SLICC", "VSG (mm)", "HCQ use (mg/day)"] if c in df_clean.columns]

# % de nulos tras 5c
null_perc_after_corr = (df_clean.isna().sum() / len(df_clean) * 100)

# Candidatas: numéricas, no en no_impute, 0 < % nulos ≤ 50
num_cols = [c for c in df_clean.columns if pd.api.types.is_numeric_dtype(df_clean[c])]
final_impute_cols = [
    c for c in num_cols
    if (c not in no_impute) and (0 < null_perc_after_corr.get(c, 0) <= 50)
]

# Aplicar mediana por Country
final_log = []
for c in final_impute_cols:
    n_before = int(df_clean[c].isna().sum())
    df_clean[c] = df_clean.groupby("Country")[c].transform(lambda s: s.fillna(s.median()))
    n_after = int(df_clean[c].isna().sum())
    final_log.append({"variable": c, "rellenos_final": n_before - n_after})

final_impute_report = pd.DataFrame(final_log).sort_values("rellenos_final", ascending=False)
print("Micro-imputación final — valores rellenados (top 20):")
display(final_impute_report.head(20))

# Resumen de nulos definitivo
final_nulls = (df_clean.isna().sum() / len(df_clean) * 100).round(2).sort_values(ascending=False)
print("\n% de nulos definitivo (top 15):")
display(final_nulls.head(15).to_frame(name="%_nulos"))


Micro-imputación final — valores rellenados (top 20):


,variable,rellenos_final
11,Energy intake (kcal/day),66
12,Carbohydrate intake (g/day),66
13,Protein intake (g/day),66
14,Lipid intake (g/day),65
5,Folic acid (ng/mL),64
8,Hemoglobin,40
7,Vitamin D (ng/mL),8
4,Urea (mg/dL),5
0,Time of disease (years),3
9,RDW,1



% de nulos definitivo (top 15):


,%_nulos
MCS12 (HRQoL),73.45
PCS12 (HRQoL),73.45
METs-min/week,73.45
VSG (mm),53.79
SLICC,53.79
Lipid (%TEI),51.03
Carbohydrate intake (%TEI),50.34
Protein intake (%TEI),49.31
HCQ use (mg/day),49.31
Carbohydrate intake (g/day),26.55


# 6) Exportación del dataset final (df_clean)




**Archivos exportados:**
- `dataset_ready.csv` y `dataset_ready.xlsx` → dataset definitivo, con imputaciones y correcciones aplicadas.
- `imputation_report.csv` → resumen de valores imputados.
- `outlier_summary.csv` → resumen de outliers detectados por IQR.
- `correction_report.csv` → detalle de las correcciones por rangos clínicos.

Este será el dataset base para los análisis estadísticos y la modelización.

In [15]:
from pathlib import Path

out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

# Archivos de salida
csv_final = out_dir / "dataset_ready.csv"
xlsx_final = out_dir / "dataset_ready.xlsx"
imp_csv = out_dir / "imputation_report.csv"
outliers_csv = out_dir / "outlier_summary.csv"
corrections_csv = out_dir / "correction_report.csv"

# Exportar dataset definitivo
df_clean.to_csv(csv_final, index=False)
with pd.ExcelWriter(xlsx_final, engine="openpyxl") as w:
    df_clean.to_excel(w, sheet_name="master", index=False)

# Exportar reportes (si existen, si no exporta un CSV vacío)
if "imputation_report" in locals():
    imputation_report.to_csv(imp_csv, index=False)
else:
    pd.DataFrame().to_csv(imp_csv, index=False)

if "outlier_summary" in locals():
    outlier_summary.to_csv(outliers_csv, index=False)
else:
    pd.DataFrame().to_csv(outliers_csv, index=False)

if "correction_report" in locals():
    correction_report.to_csv(corrections_csv, index=False)
else:
    pd.DataFrame().to_csv(corrections_csv, index=False)

print("✅ Exportación completada")
print("Dataset final CSV:", csv_final.resolve())
print("Dataset final XLSX:", xlsx_final.resolve())
print("Reporte imputación:", imp_csv.resolve())
print("Reporte outliers:", outliers_csv.resolve())
print("Reporte correcciones:", corrections_csv.resolve())


✅ Exportación completada
Dataset final CSV: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_ready.csv
Dataset final XLSX: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_ready.xlsx
Reporte imputación: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\imputation_report.csv
Reporte outliers: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\outlier_summary.csv
Reporte correcciones: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\correction_report.csv
